<a href="https://colab.research.google.com/github/Vespoi/tensorflow_study/blob/main/%EB%AA%A8%EB%8D%B8_%EB%A7%8E%EC%9D%B4_%EB%A7%8C%EB%93%A4%EA%B8%B0_3%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/공부(개인)/인공지능/Dacon/데이콘 Basic 음악 장르 분류 AI 경진대회/데이터/train.csv")

In [ ]:
#정답 데이터 숫자로 바꾸기
import numpy as np
from sklearn import preprocessing

y_raw = data[['genre']]

label_encoder = preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(y_raw)

데이터 저장

In [ ]:
unique = data['genre'].unique()
num_classes = len(unique)
genre_list = data['genre'].str.replace('[^a-zA-Z]','').unique()

for i, genre in enumerate(unique):
  globals()["data_{}".format(genre_list[i])] = data[data['genre'].str.contains(genre)]

Y데이터 무식하게 만들기

In [ ]:
for i in genre_list:
  globals()["Y_{}".format(i)] = []
for i in genre_list:
  for j in range(len(globals()["data_{}".format(i)])):
    if i == 'DarkTrap' : globals()["Y_{}".format(i)].append(0)
    if i == 'Emo' : globals()["Y_{}".format(i)].append(1)
    if i == 'Hiphop' : globals()["Y_{}".format(i)].append(2)
    if i == 'Pop' : globals()["Y_{}".format(i)].append(3)
    if i == 'Rap' : globals()["Y_{}".format(i)].append(4)
    if i == 'RnB' : globals()["Y_{}".format(i)].append(5)
    if i == 'TrapMetal' : globals()["Y_{}".format(i)].append(6)
    if i == 'UndergroundRap' : globals()["Y_{}".format(i)].append(7)
    if i == 'dnb' : globals()["Y_{}".format(i)].append(8)
    if i == 'hardstyle' : globals()["Y_{}".format(i)].append(9)
    if i == 'psytrance' : globals()["Y_{}".format(i)].append(10)
    if i == 'techhouse' : globals()["Y_{}".format(i)].append(11)
    if i == 'techno' : globals()["Y_{}".format(i)].append(12)
    if i == 'trance' : globals()["Y_{}".format(i)].append(13)
    if i == 'trap' : globals()["Y_{}".format(i)].append(14)

데이터 합치기

In [ ]:
#데이터 골라서 합치기
con_data1 = pd.concat([data_Pop, data_Emo], join='inner')
con_Y1 = Y_Pop + Y_Emo

In [ ]:
con_data2 = pd.concat( [data_trap, data_techno, data_hardstyle, data_techhouse, data_dnb, data_Hiphop, data_psytrance] , join='inner')
con_Y2 = Y_trap + Y_techno + Y_hardstyle + Y_techhouse + Y_dnb + Y_Hiphop + Y_psytrance

In [ ]:
con_data3 = pd.concat( [data_TrapMetal, data_RnB] , join='inner')
con_Y3 = Y_TrapMetal + Y_RnB

In [ ]:
con_data4 = pd.concat( [data_DarkTrap, data_Rap, data_UndergroundRap, data_trance]  , join='inner')
con_Y4 = Y_DarkTrap + Y_Rap + Y_UndergroundRap + Y_trance

데이터셋 만들기

In [ ]:
import tensorflow as tf

ds1 = tf.data.Dataset.from_tensor_slices( ( dict(con_data1) , con_Y1) )
ds2 = tf.data.Dataset.from_tensor_slices( ( dict(con_data2) , con_Y2) )
ds3 = tf.data.Dataset.from_tensor_slices( ( dict(con_data3) , con_Y3) )
ds4 = tf.data.Dataset.from_tensor_slices( ( dict(con_data4) , con_Y4) )

feature_columns

In [ ]:
feature_columns = []
feature_layer_inputs = {}

for header in ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',"liveness","valence","tempo"]:
  feature_columns.append(tf.feature_column.numeric_column(header))
  feature_layer_inputs[header] = tf.keras.Input(shape=(num_classes,), name=header)

모델 만들기

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
dense1 = tf.keras.layers.Dense(256, activation = 'relu')(feature_layer_outputs)
dense2= tf.keras.layers.Dense(512, activation = 'relu')(dense1)
dense3= tf.keras.layers.Dense(256, activation = 'relu')(dense2)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(dense3)

In [ ]:
model1 = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=output)
model2 = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=output)
model3 = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=output)
model4 = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=output)

학습

In [ ]:
model1.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['acc'])

ds_batch1 = ds1.batch(64)

model1.fit(ds_batch1, shuffle=True, epochs=150)

In [ ]:
model2.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['acc'])

ds_batch2 = ds2.batch(64)

model2.fit(ds_batch2, shuffle=True, epochs=150)

In [ ]:
model3.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['acc'])

ds_batch3 = ds3.batch(64)

model3.fit(ds_batch3, shuffle=True, epochs=150)

In [ ]:
model4.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['acc'])

ds_batch4 = ds4.batch(64)

model4.fit(ds_batch4, shuffle=True, epochs=200)

모델 합치기

In [ ]:
#학습 금지 설정
for i in model1.layers:
  i.tranable=False
for i in model2.layers:
  i.tranable=False
for i in model3.layers:
  i.tranable=False
for i in model4.layers:
  i.tranable=False

In [ ]:
model1_last_layer = model1.layers[-1].output
model2_last_layer = model2.layers[-1].output
model3_last_layer = model3.layers[-1].output
model4_last_layer = model4.layers[-1] .output

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)

concat = tf.keras.layers.Concatenate()([feature_layer_outputs, model1_last_layer, model2_last_layer, model3_last_layer, model4_last_layer])
output_merged = tf.keras.layers.Dense(num_classes, activation='softmax')(concat)

In [ ]:
#feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
#feature_layer_outputs = feature_layer(feature_layer_inputs)
#layer1 = model1_last_layer(feature_layer_outputs)
#layer2 = model2_last_layer(layer1)
#layer3 = model3_last_layer(layer2)
#layer4 = model4_last_layer(layer3)
#output_merged = tf.keras.layers.Dense(num_classes, activation='softmax')(layer4)

In [ ]:
merged_model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=output_merged)

In [ ]:
ds = tf.data.Dataset.from_tensor_slices( ( dict(data) , y_train) )

In [ ]:
merged_model.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['acc'])

ds_batch = ds.batch(64)

merged_model.fit(ds_batch, shuffle=True, epochs=250)

결과물 제작

In [ ]:
from sklearn import preprocessing
test = pd.read_csv('/content/drive/MyDrive/공부(개인)/인공지능/Dacon/데이콘 Basic 음악 장르 분류 AI 경진대회/데이터/test.csv')

ds_test = tf.data.Dataset.from_tensor_slices( (dict(test)))

y_raw = data[['genre']]
label_encoder = preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(y_raw)

submission = pd.read_csv("/content/drive/MyDrive/공부(개인)/인공지능/Dacon/데이콘 Basic 음악 장르 분류 AI 경진대회/데이터/sample_submission.csv")

ds_test_batch = ds_test.batch(32)

merged_model.evaluate(ds_test_batch)

predict = merged_model.predict(ds_test_batch)
print(len(predict))

count = 0
index = []

for i in range(len(predict)):
    results = np.argsort(predict[count])[::-1]
    labels = label_encoder.inverse_transform(results)
    index.append(labels[0])

    count += 1

In [ ]:
submission["genre"] = index
submission.to_csv("/content/drive/MyDrive/공부(개인)/인공지능/Dacon/데이콘 Basic 음악 장르 분류 AI 경진대회/테스트 제작/submit7.csv", index = False)